In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import datetime as dt
import cv2
import os
from collections import Counter
from tqdm import tqdm
from tensorflow import keras
# 模型文件
from function.ResNet.model import ResNetModel

# 训练参数设置

In [2]:
# 设置初始学习率
learning_rate=0.0001
'''
在此处设置后面模型使用的优化器
'''
# 设置模型训练优化器，默认为Adam
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

'''
# SGD和momentum
optimizer = keras.optimizers.SGD(learning_rate=learning_rate,momentum = 0.9)

# Adagrad
optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate, epsilon=None, decay=0.0)
'''

# 设置批处理数量
batch_size = 64
# 设置迭代周期
epoch = 30

# 设置模型保存地址
checkpoints = 'checkpoints/'

## optimizers简介

### keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

    lr: float >= 0. 学习率。
    beta_1: float, 0 < beta < 1. 通常接近于 1。
    beta_2: float, 0 < beta < 1. 通常接近于 1。
    epsilon: float >= 0. 模糊因子. 若为 None, 默认为 K.epsilon()。
    decay: float >= 0. 每次参数更新后学习率衰减值。
    amsgrad: boolean. 是否应用此算法的 AMSGrad 变种，来自论文 "On the Convergence of Adam and Beyond"。

### keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

    包含扩展功能的支持： - 动量（momentum）优化, - 学习率衰减（每次参数更新后） - Nestrov 动量 (NAG) 优化
    
    lr: float >= 0. 学习率。
    momentum: float >= 0. 参数，用于加速 SGD 在相关方向上前进，并抑制震荡。
    decay: float >= 0. 每次参数更新后学习率衰减值。
    nesterov: boolean. 是否使用 Nesterov 动量。

### keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)

    lr: float >= 0. 学习率.
    epsilon: float >= 0. 若为 None, 默认为 K.epsilon().
    decay: float >= 0. 每次参数更新后学习率衰减值.

# TFRecord数据加载

In [3]:
# TFRecord解析
def parse_function(example1):
    # 特征列
    features = tf.io.parse_single_example(example1, features={
        'data': tf.io.FixedLenFeature(shape=(), dtype=tf.string),
        'label': tf.io.FixedLenFeature(shape=(), dtype=tf.int64),
    })
    # 数据转换
    features['data'] = tf.io.decode_raw(features['data'], tf.float32)
    features['label'] = tf.cast(features['label'], dtype = tf.int32)
    # 重新reshape成3通道
    skeleton = tf.reshape(features['data'],(100,100,3))
    label = features['label']
    return skeleton, label

def load_dataset(filename,is_train):
    data_reading = tf.data.TFRecordDataset(filename)
    dataset = data_reading.map(parse_function)
    # 在缓冲区中随机打乱数据
    dataset = dataset.shuffle(buffer_size = 99999)
    # 每次加载图片张数
    dataset = dataset.batch(batch_size)
    return dataset

In [4]:
# 加载训练数据
training_filename = 'data/train_data.tfrecords'
data_train = load_dataset(training_filename,True)
# 加载测试数据
test_filename = 'data/test_data.tfrecords'
data_test = load_dataset(test_filename,False)

In [5]:
# 获取图片shape
for item in data_train.take(1):
    w = item[0].shape[1]
    h = item[0].shape[2]
    c = item[0].shape[3]

# 模型定义

In [6]:
# 获取label数量
my_class_path = 'data/trainlabel_list.npy'
my_class = np.load(my_class_path)
label_counts = len(my_class)
# 加载模型结构
Resnetmodel = ResNetModel(input_shape=(w,h,c),classes=label_counts)
ResNet_model = Resnetmodel.ResNet50()
ResNet_model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 106, 106, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 50, 50, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 50, 50, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

# 模型训练

In [7]:
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard)

# 编译模型来配置学习过程
ResNet_model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
callbacks = [
#     ReduceLROnPlateau(verbose=1),
    # 提前结束解决过拟合
    # EarlyStopping(patience=10, verbose=1),
    # 保存模型
    ModelCheckpoint(checkpoints + 'resnet_train_{epoch}.tf', monitor='accuracy',verbose=0,
                    # 当设置为True时，将只保存在验证集上性能最好的模型
                    save_best_only=True, save_weights_only=True,
                    # CheckPoint之间的间隔的epoch数
                    period=2),
    TensorBoard(log_dir='logs')
]
# 模型训练
history = ResNet_model.fit(data_train, epochs = epoch,callbacks=callbacks,validation_data = data_test)

Epoch 1/30
103/103 [==============================] - 53s 519ms/step - loss: 1.0101 - accuracy: 0.6687 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/30
103/103 [==============================] - 34s 332ms/step - loss: 0.0933 - accuracy: 0.9755 - val_loss: 7.7075 - val_accuracy: 0.0752
Epoch 3/30
103/103 [==============================] - 33s 319ms/step - loss: 0.0272 - accuracy: 0.9921 - val_loss: 12.0541 - val_accuracy: 0.0752
Epoch 4/30
103/103 [==============================] - 34s 334ms/step - loss: 0.0146 - accuracy: 0.9960 - val_loss: 13.9120 - val_accuracy: 0.0752
Epoch 5/30
103/103 [==============================] - 34s 326ms/step - loss: 0.0086 - accuracy: 0.9971 - val_loss: 10.2098 - val_accuracy: 0.1495
Epoch 6/30
103/103 [==============================] - 33s 318ms/step - loss: 0.0120 - accuracy: 0.9956 - val_loss: 1.8085 - val_accuracy: 0.6842
Epoch 7/30
103/103 [==============================] - 33s 317ms/step - loss: 0.0213 - accuracy: 0.9918 - val_loss: 0.62

In [11]:
#保存模型
ResNet_model.save_weights('checkpoints/resnet_train_30.tf')

    使用Tensorboard实时查看训练结果
    
    1.cd 到程序main目录放置的位置
    
    cd D:\Work\JupyterWorkSpace\DaYe\210428

    2.输入：tensorboard --logdir="logs"